In [1]:
import numpy as np
import torch
torch.set_printoptions(edgeitems=2, threshold=50, linewidth=75)

In [2]:
bikes_numpy = np.loadtxt(
    "https://raw.githubusercontent.com/duybluemind1988/dlwpt-code/master/data/p1ch4/bike-sharing-dataset/hour-fixed.csv", 
    dtype=np.float32, 
    delimiter=",", 
    skiprows=1, 
    converters={1: lambda x: float(x[8:10])}) # <1>
bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [3]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

We might want to break up the two-year dataset into wider observation periods, like
days. This way we’ll have N (for number of samples) collections of C sequences of length
L. In other words, our time series dataset would be a tensor of dimension 3 and shape
N × C × L. The C would remain our 17 channels, while L would be 24: 1 per hour of
the day. There’s no particular reason why we must use chunks of 24 hours, though the
general daily rhythm is likely to give us patterns we can exploit for predictions. We
could also use 7 × 24 = 168 hour blocks to chunk by week instead, if we desired. All of
this depends, naturally, on our dataset having the right size—the number of rows must
be a multiple of 24 or 168. Also, for this to make sense, we cannot have gaps in the
time series

As you learned in the previous chapter, calling view on a tensor returns a new tensor that changes the number of dimensions and the striding information, without
changing the storage. This means we can rearrange our tensor at basically zero cost,
because no data will be copied. Our call to view requires us to provide the new shape
for the returned tensor. We use -1 as a placeholder for “however many indexes are
left, given the other dimensions and the original number of elements.

In [6]:
bikes.shape[1]

17

In [4]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes

tensor([[[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
         [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
         ...,
         [2.3000e+01, 1.0000e+00,  ..., 1.7000e+01, 2.8000e+01],
         [2.4000e+01, 1.0000e+00,  ..., 2.4000e+01, 3.9000e+01]],

        [[2.5000e+01, 2.0000e+00,  ..., 1.3000e+01, 1.7000e+01],
         [2.6000e+01, 2.0000e+00,  ..., 1.6000e+01, 1.7000e+01],
         ...,
         [4.6000e+01, 2.0000e+00,  ..., 9.0000e+00, 9.0000e+00],
         [4.7000e+01, 2.0000e+00,  ..., 8.0000e+00, 8.0000e+00]],

        ...,

        [[1.7332e+04, 3.0000e+01,  ..., 4.1000e+01, 4.1000e+01],
         [1.7333e+04, 3.0000e+01,  ..., 2.7000e+01, 2.8000e+01],
         ...,
         [1.7354e+04, 3.0000e+01,  ..., 3.0000e+01, 3.6000e+01],
         [1.7355e+04, 3.0000e+01,  ..., 3.9000e+01, 4.9000e+01]],

        [[1.7356e+04, 3.1000e+01,  ..., 3.0000e+01, 3.4000e+01],
         [1.7357e+04, 3.1000e+01,  ..., 1.3000e+01, 1.9000e+01],
         ...,
         [1.73

For daily_bikes, the stride is telling us that advancing by 1 along the hour dimension (the second dimension) requires us to advance by 17 places in the storage (or
one set of columns); whereas advancing along the day dimension (the first dimension) requires us to advance by a number of elements equal to the length of a row in
the storage times 24 (here, 408, which is 17 × 24)

In [5]:
daily_bikes.shape, daily_bikes.stride()
# 24 * 17 = 408

(torch.Size([730, 24, 17]), (408, 17, 1))

We see that the rightmost dimension is the number of columns in the original
dataset. Then, in the middle dimension, we have time, split into chunks of 24 sequential hours. In other words, we now have N sequences of L hours in a day, for C channels. To get to our desired N × C × L ordering, we need to transpose the tensor:

In [7]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [9]:
first_day = bikes[:24].long()
print(first_day.shape)
first_day

torch.Size([24, 17])


tensor([[ 1,  1,  ..., 13, 16],
        [ 2,  1,  ..., 32, 40],
        ...,
        [23,  1,  ..., 17, 28],
        [24,  1,  ..., 24, 39]])

In [11]:
weather_onehot = torch.zeros(first_day.shape[0], 4)
print(weather_onehot.shape)
weather_onehot

torch.Size([24, 4])


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

The “weather situation” variable is ordinal. It has four levels: 1 for good weather, and 4
for, er, really bad. We could treat this variable as categorical, with levels interpreted as
labels, or as a continuous variable

In [12]:
first_day[:,9] # wether column

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2,
        2, 2])

In [13]:
weather_onehot.scatter_(
    dim=1, 
    index=first_day[:,9].unsqueeze(1).long() - 1, # <1>
    value=1.0)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [17]:
print(bikes[:24].shape)
bikes[:24]

torch.Size([24, 17])


tensor([[ 1.,  1.,  ..., 13., 16.],
        [ 2.,  1.,  ..., 32., 40.],
        ...,
        [23.,  1.,  ..., 17., 28.],
        [24.,  1.,  ..., 24., 39.]])

Last, we concatenate our matrix to our original dataset using the cat function.
Let’s look at the first of our results:

In [18]:
print(torch.cat((bikes[:24], weather_onehot), 1).shape)
torch.cat((bikes[:24], weather_onehot), 1)

torch.Size([24, 21])


tensor([[ 1.,  1.,  ...,  0.,  0.],
        [ 2.,  1.,  ...,  0.,  0.],
        ...,
        [23.,  1.,  ...,  0.,  0.],
        [24.,  1.,  ...,  0.,  0.]])

In [14]:
torch.cat((bikes[:24], weather_onehot), 1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,
          6.0000,  0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,
          3.0000, 13.0000, 16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

Here we prescribed our original bikes dataset and our one-hot-encoded “weather situation” matrix to be concatenated along the column dimension (that is, 1). In other
words, the columns of the two datasets are stacked together; or, equivalently, the new
one-hot-encoded columns are appended to the original dataset. For cat to succeed, it
is required that the tensors have the same size along the other dimensions—the row
dimension, in this case. Note that our new last four columns are 1, 0, 0, 0, exactly
as we would expect with a weather value of 1.
We could have done the same with the reshaped daily_bikes tensor. Remember
that it is shaped (B, C, L), where L = 24. We first create the zero tensor, with the same
B and L, but with the number of additional columns as C:

In [19]:
daily_bikes.shape # 730 days, 17 features each day, 24 hour each features

torch.Size([730, 17, 24])

In [20]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4,
                                   daily_bikes.shape[2])
print(daily_weather_onehot.shape)
daily_weather_onehot

torch.Size([730, 4, 24])


tensor([[[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        [[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        ...,

        [[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        [[0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]]])

In [21]:
daily_weather_onehot.scatter_(
    1, daily_bikes[:,9,:].long().unsqueeze(1) - 1, 1.0)
print(daily_weather_onehot.shape)
daily_weather_onehot

torch.Size([730, 4, 24])


tensor([[[1., 1.,  ..., 0., 0.],
         [0., 0.,  ..., 1., 1.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        [[0., 0.,  ..., 1., 1.],
         [1., 1.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        ...,

        [[0., 0.,  ..., 1., 1.],
         [1., 1.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]],

        [[1., 1.,  ..., 1., 1.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.],
         [0., 0.,  ..., 0., 0.]]])

In [22]:
print(daily_bikes.shape)
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)
print(daily_bikes.shape)
daily_bikes

torch.Size([730, 17, 24])
torch.Size([730, 21, 24])


tensor([[[1.0000e+00, 2.0000e+00,  ..., 2.3000e+01, 2.4000e+01],
         [1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00]],

        [[2.5000e+01, 2.6000e+01,  ..., 4.6000e+01, 4.7000e+01],
         [2.0000e+00, 2.0000e+00,  ..., 2.0000e+00, 2.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00]],

        ...,

        [[1.7332e+04, 1.7333e+04,  ..., 1.7354e+04, 1.7355e+04],
         [3.0000e+01, 3.0000e+01,  ..., 3.0000e+01, 3.0000e+01],
         ...,
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00]],

        [[1.7356e+04, 1.7357e+04,  ..., 1.7378e+04, 1.7379e+04],
         [3.1000e+01, 3.1000e+01,  ..., 3.1000e+01, 3.1000e+01],
         ...,
         [0.00

We mentioned earlier that this is not the only way to treat our “weather situation” variable. Indeed, its labels have an ordinal relationship, so we could pretend they are special values of a continuous variable. We could just transform the variable so that it runs
from 0.0 to 1.0:

In [24]:
print(daily_bikes[:, 9, :] .shape)
daily_bikes[:, 9, :] 

torch.Size([730, 24])


tensor([[1., 1.,  ..., 2., 2.],
        [2., 2.,  ..., 1., 1.],
        ...,
        [2., 2.,  ..., 1., 1.],
        [1., 1.,  ..., 1., 1.]])

In [25]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0
print(daily_bikes[:, 9, :] .shape)
daily_bikes[:, 9, :] 

torch.Size([730, 24])


tensor([[0.0000, 0.0000,  ..., 0.3333, 0.3333],
        [0.3333, 0.3333,  ..., 0.0000, 0.0000],
        ...,
        [0.3333, 0.3333,  ..., 0.0000, 0.0000],
        [0.0000, 0.0000,  ..., 0.0000, 0.0000]])

There are multiple possibilities for rescaling variables. We can either map their
range to [0.0, 1.0]

In [26]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min)
                         / (temp_max - temp_min))

In [27]:
temp = daily_bikes[:, 10, :]
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - torch.mean(temp))
                         / torch.std(temp))